In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.decomposition import PCA
import sklearn as skl

# For loading huggingface stuff
from sentence_transformers import SentenceTransformer

# Feature Extraction

In [18]:
# Possible values: labse, bert, word2vec
FEATURE_EXTRACTOR = "labse" 

In [95]:
# Load the dataset from the corresponding csv

DATASET_PATHS = ["training", "validation", "test"]


In [96]:
# Load the feature extractor specified in embedding generator
feature_extractor = None 

if FEATURE_EXTRACTOR == "labse":
    # LaBSE found here: https://huggingface.co/sentence-transformers/LaBSE
    feature_extractor = SentenceTransformer('sentence-transformers/LaBSE')
else: 
    pass 

In [97]:
for db_path in DATASET_PATHS:
    loaded_df = pd.read_csv(db_path + ".csv")

    # Prepare sentence list
    sentences = loaded_df["text"]

    # Apply the feature extractor to a target dataset

    encodings = []

    if FEATURE_EXTRACTOR == "labse":
        encodings = feature_extractor.encode(sentences)
    else: 
        pass

    # Store individual floats in the vector in their own columns.

    transposed = encodings.T
    new_df = loaded_df.copy(deep=True)

    for i in range(0, len(encodings[0])):
        new_df['_e' + str(i)] = transposed[i]

    # Save the feature list to a new dataset
    NEW_FILE_PATH = db_path + "_" + FEATURE_EXTRACTOR
    new_df.to_csv(NEW_FILE_PATH + ".csv", index=False)

C:\Users\Joaquin\AppData\Local\Temp\ipykernel_27884\821129340.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['_e' + str(i)] = transposed[i]
C:\Users\Joaquin\AppData\Local\Temp\ipykernel_27884\821129340.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['_e' + str(i)] = transposed[i]
C:\Users\Joaquin\AppData\Local\Temp\ipykernel_27884\821129340.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi